In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from unicodedata import normalize
import emoji
from textblob import TextBlob
from nltk.corpus import stopwords
import wordsegment as ws
ws.load()
import contractions

In [ ]:
!pip install contractions

In [ ]:
train_df = pd.read_csv('../input/nlp-getting-started/train.csv')
test_df = pd.read_csv('../input/nlp-getting-started/test.csv')

In [ ]:
train_df.drop('id', axis=1, inplace=True)
test_df.drop('id', axis=1, inplace=True)

In [ ]:
df = train_df.append(test_df)
df

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.describe(include='all')

In [ ]:
df['keyword'].unique()

# Data Preprocessing on keyword feature

In [ ]:
df['keyword'] = df['keyword'].str.replace('%20', ' ')

In [ ]:
df.info()

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
def extract_words(tweet):
    hashtags = re.findall(r"(#\w+)", tweet)
    for hs in hashtags:
        words = " ".join(ws.segment(hs))
        tweet = tweet.replace(hs, words)
    return tweet

In [ ]:
def remove_urls(tweet):
    return re.sub(r'http\S+', '', tweet)

In [ ]:
def remove_numbers(tweet):
    return re.sub(r'[^\D\.]', '', tweet)

In [ ]:
def remove_usermentions(tweet):
    return re.sub(r'@(\w+)', '', tweet)

In [ ]:
def lowercase_words(df, index):
    text = ''

    words = df['text'][index].split(' ')
    
    for i in range(len(words)):
        if words[i] not in stop_words:
            text += words[i]
            text += ' '
    text = text.rstrip()
    df.at[index, 'text'] = text
    
    return df['text'][index]

In [ ]:
def remove_punctuations(tweet):
    return re.sub(r'[^\w\s]', '', tweet)

In [ ]:
def correct_spellings(tweet):
    return str(TextBlob(tweet).correct())

In [ ]:
for index in df.index:
    
    # remove unicode characters
    df.at[index, 'text'] = (normalize('NFKD', df['text'][index]).encode('ascii','ignore')).decode('utf-8')
    
    # remove user mentions
    df.at[index, 'text'] = remove_usermentions(df['text'][index])
    
    # remove hashtags and splits the words
    df.at[index, 'text'] = extract_words(df['text'][index])
    
    # remove contractions
    df.at[index, 'text'] = contractions.fix(df['text'][index])
    
    # convert emojis into text
    df.at[index, 'text'] = emoji.demojize(df['text'][index], delimiters=("", ""))
    
    # remove urls
    df.at[index, 'text'] = remove_urls(df['text'][index])
    
    # remove numbers
    df.at[index, 'text'] = remove_numbers(df['text'][index])
    
    # remove punctuations
    df.at[index, 'text'] = remove_punctuations(df['text'][index])
    
    # correct spelling mistakes
    df.at[index, 'text'] = correct_spellings(df['text'][index])
    
    # lower casing the words
    df.at[index, 'text'] = lowercase_words(df, index)
    
    # removing unwanted white spaces
    df.at[index, 'text'] = ' '.join(df['text'][index].split())

In [ ]:
for index in df.index:
    df.at[index, 'text'] = df['text'][index].lower()

In [ ]:
df.to_csv('nlp_tweets.csv')

In [ ]:
df